# 改进模型拟合

有一个矛盾,为了达到完美的拟合,模型必须向过拟合,才能知道哪里是边界.

面对一个机器学习的问题,最初的目标是找到一个表现出一些泛化能力的模型,然后可以过拟合.之后的的事情是对抗过拟合改善模型的泛化能力.使其能达到第一个里程碑: 模型的表现要超过简单方法的基线.

在初始训练模型时常见的 3 个问题

- 训练无效,训练损失不会随着时间而减小.
- 训练效果很好,但是模型的效果无法打败基线.
- 训练和验证效果都很好,模型效果也超过了基线,但是似乎一直没有过拟合出现.


## 调整梯度下降的参数

有时候会碰到,无论如何训练,损失总是过早的停滞了.这个时候还记得前文对随机数据的实验吗?即使是随机的数据也能训练出一个模型,当然这个模型谈不上什么泛化能力.因此这个问题一定有办法解决.

通常这样的情况发生总是梯度下降的配置问题.

- 模型初始权重分布/优化器选择/学习率/批次大小等等都互相依赖.
- 其他调整学习率和批次大小就足够了,特别是学习率.


In [5]:
from keras.datasets import mnist
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

model = keras.Sequential([
    layers.Dense(512, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer=keras.optimizers.RMSprop(1.),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images,
          train_labels,
          epochs=10,
          batch_size=128,
          validation_split=0.2)


Epoch 1/10
375/375 [==============================] - 2s 5ms/step - loss: 784.9496 - accuracy: 0.4603 - val_loss: 5.4565 - val_accuracy: 0.3444
Epoch 2/10
375/375 [==============================] - 2s 5ms/step - loss: 6.6639 - accuracy: 0.3249 - val_loss: 2.3721 - val_accuracy: 0.2243
Epoch 3/10
375/375 [==============================] - 2s 5ms/step - loss: 3.6877 - accuracy: 0.2662 - val_loss: 2.1725 - val_accuracy: 0.1973
Epoch 4/10
375/375 [==============================] - 2s 5ms/step - loss: 2.7740 - accuracy: 0.2407 - val_loss: 2.2461 - val_accuracy: 0.2333
Epoch 5/10
375/375 [==============================] - 2s 5ms/step - loss: 3.0985 - accuracy: 0.2590 - val_loss: 2.2055 - val_accuracy: 0.2407
Epoch 6/10
375/375 [==============================] - 2s 5ms/step - loss: 2.6578 - accuracy: 0.2789 - val_loss: 2.0714 - val_accuracy: 0.2631
Epoch 7/10
375/375 [==============================] - 2s 5ms/step - loss: 2.5720 - accuracy: 0.2725 - val_loss: 2.1041 - val_accuracy: 0.2575
Epoc

学习率是 1 ,无论如何训练,模型的准确度一直都在 0.2 ~ 0.3 之间.接下来我们把学习率改成 1e-2.


In [6]:
from keras.datasets import mnist
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

model = keras.Sequential([
    layers.Dense(512, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer=keras.optimizers.RMSprop(1e-2),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images,
          train_labels,
          epochs=10,
          batch_size=128,
          validation_split=0.2)

Epoch 1/10
375/375 [==============================] - 2s 5ms/step - loss: 0.3525 - accuracy: 0.9108 - val_loss: 0.1891 - val_accuracy: 0.9508
Epoch 2/10
375/375 [==============================] - 2s 5ms/step - loss: 0.1420 - accuracy: 0.9642 - val_loss: 0.1329 - val_accuracy: 0.9705
Epoch 3/10
375/375 [==============================] - 2s 5ms/step - loss: 0.1143 - accuracy: 0.9728 - val_loss: 0.1561 - val_accuracy: 0.9709
Epoch 4/10
375/375 [==============================] - 2s 5ms/step - loss: 0.1024 - accuracy: 0.9782 - val_loss: 0.1918 - val_accuracy: 0.9694
Epoch 5/10
375/375 [==============================] - 2s 5ms/step - loss: 0.0939 - accuracy: 0.9814 - val_loss: 0.2687 - val_accuracy: 0.9628
Epoch 6/10
375/375 [==============================] - 2s 5ms/step - loss: 0.0781 - accuracy: 0.9848 - val_loss: 0.2403 - val_accuracy: 0.9709
Epoch 7/10
375/375 [==============================] - 2s 5ms/step - loss: 0.0667 - accuracy: 0.9870 - val_loss: 0.2023 - val_accuracy: 0.9743
Epoch 

调整过学习率,模型训练就正常多了..

如果出现损失一直无法降低等情况

- 降低或提高学习率.过高的学习率会使更新大大超过适当的拟合范围,模型就一直在这个范围左右反复横跳.过低的学习率反而使得学习的速度非常慢,简单的几轮训练验证几乎完全不可能够到合适的拟合范围.
- 增加批次大小,批次的样本更多,会使得信息量加大,噪音相对更小.

以上,终于有一个能让训练开始的配置了.
